In [1]:
import glob
import pickle
import numpy
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [2]:
def get_notes():
    notes = []

    for file in glob.glob("dataset3/10cc/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        parsing_notes = None

        try: 
            s2 = instrument.partitionByInstrument(midi)
            parsing_notes = s2.parts[0].recurse() 
        except: 
            parsing_notes = midi.flat.notes

        for unit in parsing_notes:
            if isinstance(unit, note.Note):
                notes.append(str(unit.pitch))
            elif isinstance(unit, chord.Chord):
                notes.append('.'.join(str(n) for n in unit.normalOrder))

    with open('./models/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [3]:
def prepare_sequences(notes, n_vocab):
 
    sequence_length = 100
    name_of_pitch = sorted(set(item for item in notes))
    note_to_int = dict((note, number) for number, note in enumerate(name_of_pitch))

    inputs_model = []
    output_model = []

    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        inputs_model.append([note_to_int[char] for char in sequence_in])
        output_model.append(note_to_int[sequence_out])

    n_patterns = len(inputs_model)

    inputs_model = numpy.reshape(inputs_model, (n_patterns, sequence_length, 1))
    
    inputs_model = inputs_model / float(n_vocab)

    output_model = np_utils.to_categorical(output_model)

    return (inputs_model, output_model)

In [4]:
def create_network(inputs_model, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(inputs_model.shape[1], inputs_model.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [5]:
def train(model, inputs_model, output_model):
    filepath = "./models/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(inputs_model, output_model, epochs=200, batch_size=128, callbacks=callbacks_list)

In [6]:
def train_network():
    notes = get_notes()
    n_vocab = len(set(notes))
    inputs_model, output_model = prepare_sequences(notes, n_vocab)
    model = create_network(inputs_model, n_vocab)
    train(model, inputs_model, output_model)

In [ ]:
train_network()

In [27]:
def generate():
    with open('./models/notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    name_of_pitch = sorted(set(item for item in notes))
    n_vocab = len(set(notes))

    inputs_model, normalized_input = prepare_sequences(notes, name_of_pitch, n_vocab)
    model = create_network(normalized_input, n_vocab)
    prediction_output = generate_notes(model, inputs_model, name_of_pitch, n_vocab)
    create_midi(prediction_output)

In [28]:
def prepare_sequences(notes, name_of_pitch, n_vocab):
    note_to_int = dict((note, number) for number, note in enumerate(name_of_pitch))

    sequence_length = 100
    inputs_model = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        inputs_model.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(inputs_model)

    normalized_input = numpy.reshape(inputs_model, (n_patterns, sequence_length, 1))
    
    normalized_input = normalized_input / float(n_vocab)

    return (inputs_model, normalized_input)

In [29]:
def create_network(inputs_model, n_vocab):
    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(inputs_model.shape[1], inputs_model.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))
    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
    model.add(LSTM(512))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    model.load_weights('./models/weights-improvement-182-0.0301-bigger.hdf5')

    return model

In [30]:
def generate_notes(model, inputs_model, name_of_pitch, n_vocab):

    start = numpy.random.randint(0, len(inputs_model)-1)

    int_to_note = dict((number, note) for number, note in enumerate(name_of_pitch))

    pattern = inputs_model[start]
    prediction_output = []

    for note_index in range(500):
        prediction_input = numpy.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = numpy.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [31]:
def create_midi(prediction_output):
    offset = 0
    output_notes = []

    for pattern in prediction_output:
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
            
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')


In [ ]:
generate()